# AlphaFold RunPod Submit Helper

이 노트북은 `--upload-inputs` 옵션으로 로컬 FASTA 파일/폴더를 압축해 RunPod Serverless 엔드포인트에 전송 및 모델을 실행합니다. 아래 순서를 참고하세요.
1. (필요 시) `ipyfilechooser` 설치
2. 파일/폴더 선택 위젯 실행
3. RunPod API 정보 입력
4. Monomer 또는 Multimer 명령 셀 실행


In [ ]:
# Optional: 필요한 라이브러리 설치 (설치 후 재시작 필요)
%pip install -q ipyfilechooser


In [ ]:
# 파일 선택

from pathlib import Path
import shlex
import subprocess
import sys

import ipywidgets as widgets
from ipyfilechooser import FileChooser
from IPython.display import display

REPO_ROOT = Path('..').resolve()
CLIENT_SCRIPT = REPO_ROOT / 'client' / 'submit_job.py'
if not CLIENT_SCRIPT.exists():
    raise FileNotFoundError(f'client/submit_job.py not found at {CLIENT_SCRIPT}')

def _selected_path(chooser):
    value = getattr(chooser, 'selected_path', None)
    if value:
        return value
    return getattr(chooser, 'selected', None)

fasta_file_chooser = FileChooser(str(REPO_ROOT / 'sample_data'))
fasta_file_chooser.title = 'Select FASTA file (monomer or single-file multimer)'
fasta_file_chooser.filter_pattern = '*.fa*'

fasta_dir_chooser = FileChooser(str(REPO_ROOT / 'sample_data'))
fasta_dir_chooser.title = 'Select FASTA folder (multimer batches)'
fasta_dir_chooser.show_only_dirs = True

clear_file_button = widgets.Button(description='Clear file selection', button_style='warning')
clear_dir_button = widgets.Button(description='Clear folder selection', button_style='warning')

def _clear_file_selection(_):
    fasta_file_chooser.reset()

def _clear_dir_selection(_):
    fasta_dir_chooser.reset()

clear_file_button.on_click(_clear_file_selection)
clear_dir_button.on_click(_clear_dir_selection)

file_picker_box = widgets.VBox([fasta_file_chooser, clear_file_button])
folder_picker_box = widgets.VBox([fasta_dir_chooser, clear_dir_button])

display(file_picker_box)
display(folder_picker_box)



In [ ]:
# 위젯 설정

api_key_box = widgets.Password(
    description='API Key',
    placeholder='RUNPOD_API_KEY env or paste here',
    layout=widgets.Layout(width='50%'),
)
endpoint_box = widgets.Text(
    description='Endpoint',
    placeholder='n3tcpxdv3irr46',
    value='n3tcpxdv3irr46',
    layout=widgets.Layout(width='50%'),
)
db_preset_dropdown = widgets.Dropdown(
    description='DB preset',
    options=['full_dbs', 'reduced_dbs'],
    value='full_dbs',
)
save_archive_box = widgets.Text(
    description='Save archive',
    value='alphafold_results.tar.gz',
    layout=widgets.Layout(width='50%'),
)
poll_slider = widgets.IntSlider(description='Poll (s)', min=5, max=60, step=5, value=10)
timeout_slider = widgets.IntSlider(description='Timeout (s)', min=600, max=28800, step=600, value=28800)
extra_flags_box = widgets.Text(
    description='Extra flags',
    placeholder='Optional --extra flags',
    layout=widgets.Layout(width='70%'),
)
display(widgets.VBox([
    api_key_box,
    endpoint_box,
    db_preset_dropdown,
    save_archive_box,
    poll_slider,
    timeout_slider,
    extra_flags_box,
]))


In [4]:
# 기본 설정 (공통 인자) 및 실행 함수

def build_common_args(model_preset):
    args = [
        '--model-preset', model_preset,
        '--db-preset', db_preset_dropdown.value,
        '--poll-interval', str(poll_slider.value),
        '--timeout', str(timeout_slider.value),
        '--save-archive', save_archive_box.value,
        '--upload-inputs',
        '--insecure',
    ]
    if api_key_box.value:
        args.extend(['--api-key', api_key_box.value])
    if endpoint_box.value:
        args.extend(['--endpoint', endpoint_box.value])
    extra_flags = extra_flags_box.value.strip()
    if extra_flags:
        args.append(f'--extra-flags={extra_flags}')
    return args

def run_submit(additional_args):
    cmd = [sys.executable, str(CLIENT_SCRIPT)] + additional_args
    print('Running command:')
    print(' ', ' '.join(shlex.quote(str(part)) for part in cmd))
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    try:
        for line in process.stdout:
            print(line, end='')
    finally:
        process.wait()
    if process.returncode != 0:
        raise RuntimeError(f'submit_job.py exited with code {process.returncode}')


In [ ]:
# Monomer submission cell : 모노머 실행일때만 실행
monomer_file = _selected_path(fasta_file_chooser)
if not monomer_file:
    raise ValueError('Select a FASTA file above before running this cell.')
monomer_args = ['--fasta-path', monomer_file] + build_common_args('monomer')
run_submit(monomer_args)


In [ ]:
# Single-file multimer submission cell : 멀티머 단일 파일 실행일때만 실행
multimer_file = _selected_path(fasta_file_chooser)
if not multimer_file:
    raise ValueError('Select a FASTA file above before running this cell.')
multimer_file_args = ['--fasta-path', multimer_file] + build_common_args('multimer')
run_submit(multimer_file_args)


In [ ]:
# Multimer or batch submission cell : 멀티머 또는 배치 실행일때만 실행
multimer_dir = _selected_path(fasta_dir_chooser)
if not multimer_dir:
    raise ValueError('Select a FASTA folder above before running this cell.')
multimer_args = ['--fasta-dir', multimer_dir] + build_common_args('multimer')
run_submit(multimer_args)
